In [1]:
import tensorflow as tf
import numpy as np

In [23]:
# Array of bandits. If the result pulled is more than the bandits then you get rewards
bandits = [0.2,0,-0.2,-5] 
num_bandits = len(bandits)
def pullBandit(bandit):
    #Get a random number
    result = np.random.randn(1)
    if result > bandit:
        #return positive reward
        return 1
    else:
        #return negative reward
        return -1

In [24]:
tf.reset_default_graph()

weights = tf.Variable(tf.ones([num_bandits]))
chosen_action = tf.argmax(weights, 0)

reward_holder = tf.placeholder(shape=[1], dtype=tf.float32)
action_holder = tf.placeholder(shape=[1], dtype=tf.int32)
responsible_weight = tf.slice(weights,action_holder,[1])
loss = -(tf.log(responsible_weight)*reward_holder)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
update = optimizer.minimize(loss)


In [ ]:
total_episodes = 1000
total_reward = np.zeros(num_bandits)
e = 0.1

init = tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:
        
        if np.random.rand(1) < e:
            action = np.random.randint(num_bandits)
        else:
            action = sess.run(chosen_action)
        
        reward = pullBandit(bandits[action])
        
        _,resp,ww = sess.run([update,responsible_weight,weights], 
                             feed_dict={reward_holder:[reward],
                                        action_holder:[action]})
        
        total_reward[action] += reward
        

In [33]:
total_episodes = 1000
total_reward = np.zeros(num_bandits)
e = 0.1

init = tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:

        # Because of the type of method not always pick the correct answer but will allow ourself to explore the possibility of what will happen if we select another choice
        if np.random.rand(1) < e:
            #a random choice
            action = np.random.randint(num_bandits)
        else:
            # the choice selected by argmax
            action = sess.run(chosen_action)
        
        reward = pullBandit(bandits[action])

        
        _,resp,ww = sess.run([update,responsible_weight,weights], 
                             feed_dict={reward_holder:[reward],
                                        action_holder:[action]})
        
        total_reward[action] += reward
        if i % 50 == 0:
            print("Running reward for the " 
                  + str(num_bandits) + " bandits: " + str(total_reward)
                 )
        i +=1
        
print ("Test value: " + str(pullBandit(bandits[action]))) 

print ("The agent thinks bandit " + str(np.argmax(ww) + 1) + " is most promising...")
if np.argmax(ww) == np.argmax(-np.array(bandits)):
    print ("... and it was right!")
else:
    print ("and it was wrong")

Running reward for the 4 bandits: [ 1.  0.  0.  0.]
Running reward for the 4 bandits: [-2.  0.  8.  1.]
Running reward for the 4 bandits: [-4. -1.  8.  2.]
Running reward for the 4 bandits: [-6. -1.  2.  8.]
Running reward for the 4 bandits: [ -5.  -2.   3.  53.]
Running reward for the 4 bandits: [ -6.  -2.   7.  98.]
Running reward for the 4 bandits: [  -6.   -3.    7.  147.]
Running reward for the 4 bandits: [  -6.   -1.    7.  195.]
Running reward for the 4 bandits: [  -9.   -1.    7.  242.]
Running reward for the 4 bandits: [  -9.   -1.    7.  288.]
Running reward for the 4 bandits: [  -8.    0.    7.  334.]
Running reward for the 4 bandits: [  -8.    0.    8.  381.]
Running reward for the 4 bandits: [  -8.   -1.    8.  430.]
Running reward for the 4 bandits: [  -8.    0.    8.  479.]
Running reward for the 4 bandits: [  -7.    1.    8.  525.]
Running reward for the 4 bandits: [  -7.    0.    9.  573.]
Running reward for the 4 bandits: [  -5.    0.   10.  620.]
Running reward for t